In [1]:
#RUNNING ON ENVIRONMENT 'UNEXT'
# Config
seed = 42  # for reproducibility
training_split_ratio = 0.9  # use 90% of samples for training, 10% for testing
num_epochs = 5
# If the following values are False, the models will be downloaded and not computed
compute_histograms = False
train_whole_images = False 
train_patches = False
import uuid
from IPython.display import display
import enum
import time
import random
import multiprocessing
from pathlib import Path
from tqdm import tqdm
import torch
torch.cuda.empty_cache()

import torchvision
import torchio as tio
import torch.nn.functional as F

import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

from IPython.display import display
from tqdm.auto import tqdm

random.seed(seed)
torch.manual_seed(seed)
plt.rcParams['figure.figsize'] = 12, 6

# import pytorch_lightning as pl
import os
import time


import torch
import torchio
# import pytorch_lightning as pl
# import wandb
from pathlib import Path
from tqdm import tqdm
import numpy as np



C:\Users\Patrick\AppData\Roaming\Python\Python310\site-packages\torchvision\models\detection\anchor_utils.py:63: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:77.)
  device: torch.device = torch.device("cpu"),


In [2]:
import torchvision.transforms as T
from matplotlib import cm
import PIL
import torchvision.transforms.functional as Ft
import cv2

transform = T.ToPILImage()

In [3]:
base_transforms = tio.Compose([tio.transforms.ToCanonical(),
                   tio.RescaleIntensity((0, 1))])
augment = tio.Compose([
            tio.RandomAffine(),
            tio.RandomMotion(p=0.1),
            tio.RandomBiasField(p=0.25),
        ])
def sub_to_aug_pair(subject: tio.Subject):
    
    transformed_subject = base_transforms(subject)
    augmented_subject = augment(transformed_subject)
    
    return transformed_subject, augmented_subject

def tensor_to_disk(path_name, data):
    with open(f'{path_name}.npy', 'wb') as f:
        np.save(f, data)



In [1]:
SAVEPATH = os.getcwd()
PATH = os.getcwd()
dataset_dir = Path(PATH + "\\normdata\\") #switched from augfix due to one hot error
images_dir = dataset_dir / 'images'
labels_dir = dataset_dir / 'labels'
image_paths = sorted(images_dir.glob('*.nii.gz'))
label_paths = sorted(labels_dir.glob('*.nii.gz'))
assert len(image_paths) == len(label_paths)
dataset = []
count = 0
for (image_path, label_path) in tqdm(zip(image_paths, label_paths), total=len(image_paths)):
        norm = tio.Subject(
            sample=tio.ScalarImage(image_path),
            label =  tio.LabelMap(label_path))

        tensor_to_disk(dataset_dir + f"/labels_new/label{count}", norm['label'][tio.DATA])        
        tensor_to_disk(dataset_dir + f"/images_new/sample{count}", norm['sample'][tio.DATA])        
        count +=1
        del norm
        break
        
        


NameError: name 'Path' is not defined

In [ ]:
mem_to_new_mem()

In [ ]:
subject_list = mem_to_sub()

In [ ]:
aug, n = sub_to_aug_pair(subject_list[0])


In [ ]:

#Visualizing tensor functions
def write_video(file_path, frames, fps, grayscale=False):
    """
    Writes frames to an mp4 video file
    :param file_path: Path to output video, must end with .mp4
    :param frames: List of PIL.Image objects
    :param fps: Desired frame rate
    """
    w, h = frames[0].size
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    writer = cv2.VideoWriter(file_path, fourcc, fps, (w, h))

    for frame in frames:
        open_cv_image = np.asarray(frame)
        if not grayscale:
            # Convert RGB to BGR 
            open_cv_image = open_cv_image[:, :, ::-1]
           
        else:
            open_cv_image = cv2.cvtColor(open_cv_image, cv2.COLOR_GRAY2BGR)
        
        writer.write(open_cv_image)

    writer.release() 

def sample_to_video(inputs, name="label_vid", repeat=3):
    inputs = inputs[0]
    slices = []
    for i in range(inputs.size()[2]):
        new = inputs[:,:,i]
        pil_image = transform(new)
        for _ in range(repeat):
            slices.append(pil_image)
        
    return slices


def label_to_video(inputs, name="label_vid", repeat=3):
    """
    Given an output tensor from the network with shape [6, 256, 256, 10]), return a
    video of the tensor with name "name"
    """
    num_classes=6
    color = (torch.ones((1,inputs.shape[1],inputs.shape[2],inputs.shape[3]))/num_classes)
    re_colored = inputs * color
    re_colored = re_colored[0]
    slices = []
    magma = cm.get_cmap('YlGnBu')
    for i in range(re_colored.size()[2]):
        new = re_colored[:,:,i]
        new = magma(new)
        new = new[:,:,:3]*255
        pil_image = PIL.Image.fromarray(new.astype(np.uint8))
#         display(pil_image)
        for _ in range(repeat):
            slices.append(pil_image)
    return slices

slices=label_to_video(tens['label'])
write_video(os.getcwd() + "/m.mp4", slices, 10)

slices=sample_to_video(d[0]['sample'])
write_video(os.getcwd() + "/test_sample_vid.mp4", slices, 10, True)